Através dessa análise é possível obter diversos insights ...... Data set laranjas...

## Metodologia

### 1. Obtenção de Dados

#### 1.1 Importação de bibliotecas

In [7]:
import pandas as pd
from IPython.display import display, Markdown, HTML
import joblib
import numpy as np

from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, MinMaxScaler, OneHotEncoder, OrdinalEncoder, LabelEncoder
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import LogisticRegression

from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import ShuffleSplit, GridSearchCV, KFold, cross_validate
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier

#### 1.2 Importação dos Dados

In [41]:
caminho = '../data/raw/Orange_Quality_Data.csv'
laranjas = pd.read_csv(caminho) #Obtendo o dataset

laranjas

,Size (cm),Weight (g),Brix (Sweetness),pH (Acidity),Softness (1-5),HarvestTime (days),Ripeness (1-5),Color,Variety,Blemishes (Y/N),Quality (1-5)
0,7.5,180,12.0,3.2,2.0,10,4.0,Orange,Valencia,N,4.0
1,8.2,220,10.5,3.4,3.0,14,4.5,Deep Orange,Navel,N,4.5
2,6.8,150,14.0,3.0,1.0,7,5.0,Light Orange,Cara Cara,N,5.0
3,9.0,250,8.5,3.8,4.0,21,3.5,Orange-Red,Blood Orange,N,3.5
4,8.5,210,11.5,3.3,2.5,12,5.0,Orange,Hamlin,Y (Minor),4.5
...,...,...,...,...,...,...,...,...,...,...,...
236,8.0,194,10.9,3.6,5.0,13,1.0,Orange-Red,Tangerine,Y (Scars),5.0
237,7.4,275,8.5,3.5,5.0,20,5.0,Light Orange,Minneola (Hybrid),N,4.0
238,7.5,196,15.7,3.0,3.0,13,3.0,Deep Orange,Temple,Y (Minor Insect Damage),5.0
239,7.2,251,9.8,4.3,3.0,23,1.0,Light Orange,Moro (Blood),Y (Minor Insect Damage),3.0


In [42]:
dicionario = pd.read_csv("../data/external/dicionario.csv")
dicionario

,variavel,descrição,tipo,subtipo
0,Size,Tamanho da fruta em cm,Quantitativa,Contínua
1,Weight,Peso da fruta em g,Quantitativa,Contínua
2,Brix,Nível de doçura,Quantitativa,Contínua
3,pH,Nível de acidez em pH,Quantitativa,Contínua
4,Softness,Maciez de 1-5,Qualitativa,Ordinal
5,HarvestTime,Dias desde a colheita,Quantitativa,Discreta
6,Ripeness,Maduração de 1-5,Qualitativa,Ordinal
7,Color,Cor da laranja,Qualitativa,Nominal
8,Variety,Variedade da laranja,Qualitativa,Nominal
9,Blemishes,Defeito,Qualitativa,Nominal


- Observamos, então, que o conjunto de dados está completo e não apresenta valores ausentes.

---

### 2. Preparação de Dados

#### 2.1 Tratamento de Dados

In [43]:
# Este problema será de Classificação pois a coluna contém categoria/classe que o modelo tentará prever
# A variável coluna 'Quality (1-5)' do dataset será a variável alvo (y), ou seja, a variável que o modelo vai tentar prever.
target_column = 'Quality (1-5)'


# Esta parte seleciona as colunas do dataset que são do tipo nominal (variáveis categóricas que não possuem uma ordem intrínseca) 
# e que não são a variável alvo (target_column).
nominal_columns = (
    dicionario
    .query("subtipo == 'Nominal' and variavel != @target_column")
    .variavel
    .to_list()
)


# Aqui são selecionadas as colunas do dataset que são do tipo contínuo (variáveis numéricas que podem assumir qualquer valor dentro de um intervalo).
# As colunas selecionadas são convertidas para uma lista
continuous_columns = (
    dicionario
    .query("subtipo == 'Contínua'")
    .variavel
    .to_list()
)


# Filtrando as colunas discretas
discrete_columns = (
    dicionario
    .query("subtipo == 'Discreta'and variavel != @target_column")
    .variavel
    .to_list()
)

ordinal_columns = (
    dicionario
    .query("subtipo == 'Ordinal' and variavel != @target_column")
    .variavel
    .to_list()
)

# X é o dataframe que contém todas as colunas do dataset original, exceto a coluna alvo (target_column). 
# ele representa as features (variáveis independentes) que serão usadas para treinar o modelo.

# Y é a ou coluna que contém apenas a variável alvo que o modelo tentará prever.
X = laranjas.drop(columns=[target_column], axis=1)
y = laranjas[target_column]
nominal_columns, continuous_columns, discrete_columns, ordinal_columns

(['Color', 'Variety', 'Blemishes'],
 ['Size', 'Weight', 'Brix', 'pH'],
 ['HarvestTime'],
 ['Softness', 'Ripeness'])

In [45]:
# Tratamento de variáveis nominais (categóricas)
nominal_preprocessor = Pipeline([
    ('missing', SimpleImputer(strategy='most_frequent')),
    ('encoding', OneHotEncoder(sparse_output=False, drop='first')),
    ('normalization', StandardScaler())
])

# Tratamento de variáveis contínuas (numéricas)
continuous_preprocessor = Pipeline([
    ('missing', SimpleImputer(strategy='mean')), 
    ('normalization', StandardScaler()) 
])

# Tratamento de variáveis discretas (numéricas)
discrete_preprocessor = Pipeline([
    ('missing', SimpleImputer(strategy='mean')),
    ('normalization', StandardScaler())
])

# Tratamento de variáveis ordinais (categóricas com ordem)
ordinal_preprocessor = Pipeline([
    ('missing', SimpleImputer(strategy='most_frequent')),
    ('encoding', OrdinalEncoder()),
    ('normalization', StandardScaler())
])

# Combina todos os pipelines anteriores. 
preprocessor = ColumnTransformer([
    ('nominal', nominal_preprocessor, nominal_columns),
    ('continuous', continuous_preprocessor, continuous_columns),
    ('discrete', discrete_preprocessor, discrete_columns),
    ('ordinal', ordinal_preprocessor, ordinal_columns)
])

# Define o modelo de machine learning
model = LogisticRegression()

In [20]:
# # Tratamento de variaveis nominais (categóricas)

# nominal_preprocessor = Pipeline([
# # Criação de um pipeline para pré-processar as variáveis nominais (categóricas)
#     ('missing', SimpleImputer(strategy='most_frequent')),
#     # Substitui valores faltantes (missing) nas colunas nominais pela categoria mais frequente da coluna.
#     ('encoding', OneHotEncoder(sparse_output=False, drop='first')),
#     # Codifica variáveis categóricas em variáveis dummy. Cada categoria é transformada em uma nova coluna com valores binários (0 ou 1).
#     # A opção drop='first' remove a primeira categoria para evitar a multicolinearidade.
#     ('normalization', StandardScaler())
#     # Normaliza os dados, ajustando-os para que tenham média 0 e desvio padrão 1. 
#     # Isso é útil para métodos que assumem que os dados estão escalados, embora a normalização após a codificação OneHot não seja tão comum.
# ])


# # Tratamento de variáveis contínuas (numéricas)
# continuous_preprocessor = Pipeline([
#     ('missing', SimpleImputer(strategy='mean')), 
#     # Tratamento de dados faltantes
#     ('normalization', StandardScaler()) 
#     # Normalização de dados
# ])


# # Combina os dois pipelines anteriores. Este transformador aplica diferentes pré-processamentos a diferentes subconjuntos de colunas.
# preprocessor = ColumnTransformer([
#     ('nominal', nominal_preprocessor, nominal_columns),
#     ('continuous', continuous_preprocessor, continuous_columns)
# ])


# # Define o modelo de machine learning que será utilizado.
# # Cria um modelo de Regressão Logística, que é usado para prever uma variável binária ou multicategórica.

# model = LogisticRegression()

## Configuração do experimento

## Resultados 

## Discussão